# Basic Example

This example shows the limited code necessary to construct and analyze a simple energy system using the `assetra` package.

To clarify syntax later in the example, we setup a few helpful functions and variables. It is worth noting that the `assetra` package makes extensive use of `xarray` data structures, so some familiarity there may be helpful.

In [ ]:
from datetime import datetime 
import xarray as xr

HOURS_PER_YEAR = 8760 # h

HOURLY_DEMAND = 100 # MW
STOCHASTIC_NUM_UNITS = 110
STOCHASTIC_UNIT_NAMEPLATE_CAPACITY = 1 # MW
STOCHASTIC_UNIT_FORCED_OUTAGE_RATE = 0.05
STORAGE_NUM_UNITS = 10
STORAGE_UNIT_NAMEPLATE_CAPACITY = 1 # MW
STORAGE_UNIT_ENERGY_CAPACITY = 1 # MWh
STORAGE_UNIT_EFFICIENCY = 0.8
SIMULATION_START_HOUR = "2019-01-01 00:00:00"
SIMULATION_END_HOUR = "2019-12-31 23:00:00"
SIMULATION_TRIAL_SIZE = 100

def get_hourly_time_series_xr(
    hourly_data: list[float], 
    start_hour: datetime="2019-01-01 00:00:00"
) -> xr.DataArray:
    '''Return formatted xarray data array for a sequence of hourly datapoints

    Args:
        hourly_data (list[float]): Input data stored as consecutive hour-scale datapoints.
        start_hour (_type_, optional): Time stamp corresponding to the first datapoint.
            Defaults to "2016-01-01 00:00:00".

    Returns:
        xr.Dataarray: Formatted one-dimensional xarray data with datetime-indexed time series.
    '''
    return xr.DataArray(
        data=[float(x) for x in hourly_data],
        coords=dict(
            time=xr.date_range(start_hour, freq='1H', periods=len(hourly_data))
        )
    )


We use the `EnergySystemBuilder` to add energy units to our system.

In [ ]:
from assetra.system import EnergySystemBuilder
from assetra.units import DemandUnit, StochasticUnit, StorageUnit

builder = EnergySystemBuilder()

# add demand (constant 100 MW for one year)
new_unit = DemandUnit(
    id=0,
    hourly_demand=get_hourly_time_series_xr([HOURLY_DEMAND]*HOURS_PER_YEAR)
)
builder.add_unit(new_unit)

# add 100 (1 MW) stochastic units
for _ in range(STOCHASTIC_NUM_UNITS):
    new_unit = StochasticUnit(
        id=builder.size,
        nameplate_capacity=STOCHASTIC_UNIT_NAMEPLATE_CAPACITY,
        hourly_capacity=get_hourly_time_series_xr([STOCHASTIC_UNIT_NAMEPLATE_CAPACITY]*HOURS_PER_YEAR),
        hourly_forced_outage_rate=get_hourly_time_series_xr([STOCHASTIC_UNIT_FORCED_OUTAGE_RATE]*HOURS_PER_YEAR)
    )
    builder.add_unit(new_unit)

# add 10 (1 MW, 1 Hour) storage units
for _ in range(STORAGE_NUM_UNITS):
    new_unit = StorageUnit(
        id=builder.size,
        nameplate_capacity=STORAGE_UNIT_NAMEPLATE_CAPACITY,
        charge_rate=STORAGE_UNIT_NAMEPLATE_CAPACITY,
        discharge_rate=STORAGE_UNIT_NAMEPLATE_CAPACITY,
        charge_capacity=STORAGE_UNIT_ENERGY_CAPACITY,
        roundtrip_efficiency=STORAGE_UNIT_EFFICIENCY
    )
    builder.add_unit(new_unit)


We compile our energy system using `builder.build()`. Note the type of `energy_system` is `EnergySystem` not `EnergySystemBuilder`.

In [ ]:
energy_system = builder.build()

At this point, we have the option to save or load our energy system to the file system.

In [ ]:
from assetra.system import EnergySystem
from pathlib import Path

saved_system_dir = Path('saved_energy_system')

if not saved_system_dir.exists():
    # save energy system 
    energy_system.save(saved_system_dir)
else:
    energy_system = EnergySystem()
    energy_system.load(saved_system_dir)


We can run a probabilistic Monte Carlo simulation of our energy system for a given time frame.

In [ ]:
from assetra.simulation import ProbabilisticSimulation

simulation = ProbabilisticSimulation(
    SIMULATION_START_HOUR,
    SIMULATION_END_HOUR,
    SIMULATION_TRIAL_SIZE
)
simulation.assign_energy_system(energy_system)
simulation.run()

Finally, we evaluate the resource adequacy of our system, in this case loss of load hours.

In [ ]:
from assetra.metrics import LossOfLoadHours

lolh_model = LossOfLoadHours(simulation)
lolh = lolh_model.evaluate()

print("System LOLH:", lolh)

The purpose of this example is only to show the basic syntax needed to work with the `assetra` package. See the next [example]() for more detailed analysis of the California ISO balancing authority.